In [20]:
import geopy
import nominatim
from geopy.geocoders import Nominatim
import pandas as pd
import geopandas as gpd
from geopy.extra.rate_limiter import RateLimiter
import ssl
import certifi

In [25]:
#Use CA bundle used by requests library | Info: geopy docs -> default options context -> default_ssl_context
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

geolocator = Nominatim(user_agent="soccerGeo")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [26]:
csv = "From Where Data - Bundesliga.csv"
df = pd.read_csv(csv)
df.head()

,Player,National Team,Club,Position,Birthplace
0,Stefan Ortega,Germany,Arminia Bielefeld,GK,"Calden,Germany"
1,Stafanos Kapino,Greece,Arminia Bielefeld,GK,"Piraus,Greece"
2,Arne Schulz,Germany,Arminia Bielefeld,GK,"Bunde,Germany"
3,Amos Pieper,Germany,Arminia Bielefeld,D,"Ludinghausen,Germany"
4,Joakim Nilsson,Sweden,Arminia Bielefeld,D,"Harnosand,Sweden"


In [28]:
##Geocode and create point
df['location'] = df['Birthplace'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df.head()

,Player,National Team,Club,Position,Birthplace,location,point
0,Stefan Ortega,Germany,Arminia Bielefeld,GK,"Calden,Germany","(Calden, Landkreis Kassel, Hessen, 34379, Deut...","(51.401225249999996, 9.400978923625331, 0.0)"
1,Stafanos Kapino,Greece,Arminia Bielefeld,GK,"Piraus,Greece","(Πειραιάς, Δήμος Πειραιώς, Περιφερειακή Ενότητ...","(37.9431594, 23.6470593, 0.0)"
2,Arne Schulz,Germany,Arminia Bielefeld,GK,"Bunde,Germany","(Bunde, Landkreis Leer, Niedersachsen, 26831, ...","(53.184017, 7.2695448, 0.0)"
3,Amos Pieper,Germany,Arminia Bielefeld,D,"Ludinghausen,Germany","(Lüdinghausen, Kreis Coesfeld, Nordrhein-Westf...","(51.7717757, 7.444639, 0.0)"
4,Joakim Nilsson,Sweden,Arminia Bielefeld,D,"Harnosand,Sweden","(Härnösand, Härnösands domkyrkodistrikt, Härnö...","(62.6323084, 17.9379914, 0.0)"


In [30]:
##Separate rows that didn't geolocate to ungeolocated df   
dfValid = df.loc[df.location.notnull()].reset_index(drop=True)
dfNull = df.loc[df.location.isnull()].reset_index(drop=True)

##Create lat, lon, z from point
dfValid[['lat','lon','z']] = pd.DataFrame(dfValid['point'].tolist(), index=dfValid.index)

##Drop uneeded columns
dfValid = dfValid.drop(['location','point','z'], axis=1)
dfNull = dfNull.drop(['location','point'], axis=1)

##Concatenate geolocated and ungeolocated df's
finaldf = pd.concat([dfValid,dfNull],axis=0)

finaldf.head()

,Player,National Team,Club,Position,Birthplace,lat,lon
0,Stefan Ortega,Germany,Arminia Bielefeld,GK,"Calden,Germany",51.401225,9.400979
1,Stafanos Kapino,Greece,Arminia Bielefeld,GK,"Piraus,Greece",37.943159,23.647059
2,Arne Schulz,Germany,Arminia Bielefeld,GK,"Bunde,Germany",53.184017,7.269545
3,Amos Pieper,Germany,Arminia Bielefeld,D,"Ludinghausen,Germany",51.771776,7.444639
4,Joakim Nilsson,Sweden,Arminia Bielefeld,D,"Harnosand,Sweden",62.632308,17.937991
